# Modeling, Prediction & Evaluation

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split as tts
from sklearn.linear_model import LinearRegression as LinReg
from sklearn.linear_model import Ridge, Lasso
from sklearn.linear_model import SGDRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.tree import plot_tree
from sklearn import metrics
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

In [2]:
df = pd.read_csv("../Data/diamonds_train_clean.csv")

In [3]:
df.head()

,carat,cut,color,clarity,depth,table,price
0,1.14,5,4,6,61.0,56.0,9013
1,0.76,5,3,4,62.7,57.0,2692
2,0.84,5,4,5,61.4,56.0,4372
3,1.55,5,3,5,62.0,57.0,13665
4,0.30,5,4,2,61.9,57.0,422


In [4]:
X = df.drop(columns = 'price', axis = 1)

In [5]:
X.shape

(40455, 6)

In [6]:
y = df.price

In [7]:
y.shape

(40455,)

In [8]:
X_train, X_test, y_train, y_test = tts(X,y, test_size=0.2)

### 1. Trying different models to find the best one

In [9]:
models = {
    'lin': LinReg(),
    'ridge': Ridge(),
    'lasso': Lasso(),
    'sgd': SGDRegressor(),
    'knn': KNeighborsRegressor(),
    'grad': GradientBoostingRegressor(),
    'rfc': RandomForestRegressor()
}

In [10]:
for name, model in models.items():
    print("TRAINING: ", name)
    model.fit(X_train, y_train)

TRAINING:  lin
TRAINING:  ridge
TRAINING:  lasso
TRAINING:  sgd
TRAINING:  knn
TRAINING:  grad
TRAINING:  rfc


In [11]:
for name, model in models.items():
    y_pred = model.predict(X_test)
    print(f"------{name}------")
    print('MAE - ', metrics.mean_absolute_error(y_test, y_pred))
    print('MSE - ', metrics.mean_squared_error(y_test, y_pred))
    print('RMSE - ', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
    print('R2 - ', metrics.r2_score(y_test, y_pred))

------lin------
MAE -  854.0301382258803
MSE -  1540452.229691242
RMSE -  1241.1495597595167
R2 -  0.9019302837206742
------ridge------
MAE -  853.9462350245732
MSE -  1540414.62770735
RMSE -  1241.1344116200107
R2 -  0.9019326775734797
------lasso------
MAE -  853.5853549850069
MSE -  1540336.554636705
RMSE -  1241.102958918681
R2 -  0.9019376479346111
------sgd------
MAE -  13272334.285886033
MSE -  266553823607877.4
RMSE -  16326476.15402287
R2 -  -16969598.803580727
------knn------
MAE -  1104.8728463725126
MSE -  3577394.810871339
RMSE -  1891.4002249316084
R2 -  0.7722525324971593
------grad------
MAE -  336.0903802043096
MSE -  360571.1756657673
RMSE -  600.4757910738512
R2 -  0.9770449792505852
------rfc------
MAE -  280.8482445641654
MSE -  290029.6652668504
RMSE -  538.5440235179018
R2 -  0.9815358591217017


### 2. Now I try with Decision Tree

In [12]:
model = DecisionTreeRegressor(max_depth=10)

In [13]:
model.fit(X_train, y_train)

DecisionTreeRegressor(max_depth=10)

In [14]:
y_pred = model.predict(X_test)

In [15]:
metrics.mean_squared_error(y_train, model.predict(X_train)).round(3)

284059.274

The model with the lowest RMSE is Random Forest Regressor, so I will continue with that model

### 3. Applying the model to the test dataset

In [16]:
test = pd.read_csv("../Data/diamonds_test_clean.csv")

In [17]:
test.head()

,carat,cut,color,clarity,depth,table
0,1.00,1,1,2,56.3,64.0
1,0.83,4,4,3,62.3,58.0
2,1.00,1,6,2,67.0,53.0
3,1.00,1,3,2,66.5,62.0
4,1.20,3,2,3,62.6,57.0


I apply the RandomForestRegressor model as it is the one with the lowest RMSE from all the models tried.

In [18]:
forest = RandomForestRegressor()

In [19]:
forest.fit(X_train, y_train)

RandomForestRegressor()

In [20]:
y_pred = forest.predict(X_test)

In [21]:
print('RMSE: ', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

RMSE:  539.9212069176868


To improve my results, I will use GridSearchCV to find the best parameters in order to reduce a little bit more my RMSE

In [22]:
params = {'max_depth': [10, 100],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 3],
 'n_estimators': [100, 150, 200]}


In [23]:
grid = GridSearchCV(forest, params, verbose=1)

In [24]:
grid.fit(X_train, y_train)

Fitting 5 folds for each of 36 candidates, totalling 180 fits


GridSearchCV(estimator=RandomForestRegressor(),
             param_grid={'max_depth': [10, 100],
                         'max_features': ['auto', 'sqrt'],
                         'min_samples_leaf': [1, 2, 3],
                         'n_estimators': [100, 150, 200]},
             verbose=1)

In [25]:
print(grid.best_params_)

{'max_depth': 100, 'max_features': 'auto', 'min_samples_leaf': 3, 'n_estimators': 200}


Now that I have my best parameters, I will try the model again with those ones.

In [26]:
forest = RandomForestRegressor(max_depth = 100, min_samples_leaf = 3, n_estimators = 200)

In [27]:
forest.fit(X_train, y_train)

RandomForestRegressor(max_depth=100, min_samples_leaf=3, n_estimators=200)

In [28]:
y_pred = forest.predict(X_test)

In [29]:
print('RMSE: ', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

RMSE:  531.0328112840793


In [30]:
model2 = forest.fit(X,y)

Finally, time to apply this model to my test dataset

In [31]:
price = model2.predict(test)

In [32]:
test['price'] = price

In [33]:
test.head()

,carat,cut,color,clarity,depth,table,price
0,1.00,1,1,2,56.3,64.0,3277.803645
1,0.83,4,4,3,62.3,58.0,3051.102881
2,1.00,1,6,2,67.0,53.0,3307.381165
3,1.00,1,3,2,66.5,62.0,3174.650066
4,1.20,3,2,3,62.6,57.0,5385.914195


In [35]:
test2 = test.drop(['carat', 'cut', 'color', 'clarity', 'depth', 'table'], axis=1)

In [36]:
test2.index.rename('id', inplace=True)

In [37]:
test2.sample()

,price
id,
238,3688.908729


In [38]:
test2.to_csv('../Data/predictions.csv')